In [1]:
suppressPackageStartupMessages(require(tidyverse))
suppressPackageStartupMessages(require(data.table))


In [2]:
phe_dir <- '@@@@@@/users/ytanigaw/repos/rivas-lab/biomarkers/snpnet/disease_outcome/phe'

In [3]:
phe_names <- c(
    fread(file.path(phe_dir, 'phe.lst'), head=F) %>% pull()
#     , 'INI22402'
)

In [4]:
phe_names

[1] "HC171"    "HC188"    "HC294"    "HC299"    "HC326"    "INI22402"
 [7] "HC215"    "INI4079"  "INI4080"  "FH1065"   "HC165"    "HC221"   
[13] "HC132"    "HC225"    "HC328"

In [5]:
covars <- c(c('split', 'age', 'sex', 'Array'), paste0('PC', 1:4))

In [6]:
# phe_f <- '@@@@@@@@/ukbb24983/phenotypedata/master_phe/master.20190509.phe'
phe_f <- '@@@@@@/ukbb24983/phenotypedata/master_phe/master.20191219.phe'

phe_df <- fread(phe_f, select=c(c('FID', 'IID'), covars, phe_names), colClasses = c("FID" = "character", "IID" = "character"))

phe_df %>% dim()

[1] 516884     25

In [7]:
t2d_f <- '@@@@@@/projects/biomarkers/gwas/1000g/T2D_Eastwood_Definition.phe'

t2d_HbA1c_39_f <- '@@@@@@/projects/biomarkers/gwas/1000g/T2D_Eastwood_Definition_Excluding_Controls_HbA1c_39.phe'


In [8]:
t2d <- fread(t2d_f, colClasses = c("FID" = "character", "IID" = "character"))
t2d_HbA1c_39 <- fread(t2d_HbA1c_39_f, colClasses = c("FID" = "character", "IID" = "character"))

In [9]:
head(t2d_HbA1c_39, 0)

FID,IID,T2D
<chr>,<chr>,<int>


In [10]:
merged_df <- phe_df %>% 
left_join(t2d, by=c('FID', 'IID')) %>%
left_join(t2d_HbA1c_39 %>% rename('T2D_HbA1c_39' = 'T2D'), by=c('FID', 'IID')) %>%
replace_na(list(T2D='-9', T2D_HbA1c_39='-9'))

In [11]:
merged_df %>% dim()

[1] 516884     27

In [12]:
merged_df %>% filter(FID == '3e+06') %>% gather(key, val, -FID, -IID)

FID,IID,key,val
<chr>,<chr>,<chr>,<chr>


In [13]:
for( c in c(phe_names, 'T2D', 'T2D_HbA1c_39')){
    merged_df[[c]] <- na_if(merged_df[[c]], -9)
}


In [19]:
merged_df %>% count(T2D)

T2D,n
<chr>,<int>
1,411877
2,23842
NA,81165


In [20]:
merged_df %>% filter(FID != '3e+06') %>%
fwrite(file.path(phe_dir, 'phe.tsv'), sep='\t')


In [22]:
merged_df %>% count(split)

split,n
<chr>,<int>
test,67430
train,236004
val,33716
NA,179734


In [21]:
merged_df %>% count(T2D, split)

T2D,split,n
<chr>,<chr>,<int>
1,test,55764
1,train,195320
1,val,27917
1,NA,132876
2,test,2855
2,train,10184
2,val,1471
2,NA,9332
NA,test,8811


In [23]:
10184 + 195320

[1] 205504